## Zero-Shot Image Classification Inference using Online Endpoints

This sample shows how to deploy `zero-shot-image-classification` type models to an online endpoint for inference.

### Task
`zero-shot-image-classification` takes in images and a set of user provided labels. For each image, probabilities are assigned corresponding to the potential labels.
Zero-Shot image classification uses a pre-trained model to run inference on data with classes that were not used during training.

### Model
Models that can perform the `zero-shot-image-classification` task are tagged with `zero-shot-image-classification`. We will use the `openai/clip-vit-base-patch32` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import_model_into_registry.ipynb) and then use them for inference.

### Inference data
We will use the [fridgeObjects](https://cvbp-secondary.z19.web.core.windows.net/datasets/image_classification/fridgeObjects.zip) dataset.


### Outline
1. Setup pre-requisites
2. Pick a model to deploy
3. Prepare data for inference
4. Deploy the model to an online endpoint for real time inference
5. Test the endpoint
6. Clean up resources - delete the online endpoint

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name="azureml",
)
# Generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

### 5. Test the endpoint

We will fetch some sample data from the test dataset and submit to online endpoint for inference.

In [ ]:
import base64
import json
import os

dataset_dir = "./clip-large-inference-benchmarking/"

sample_image = os.path.join(dataset_dir, "1.jpg")
# sample_image = os.path.join(dataset_dir, "beach_large.jpg")
# sample_image = os.path.join(dataset_dir, "beach_huge.jpg")
labels = "water_bottle, milk_bottle, carton, can"


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()


request_json = {
    "input_data": {
        "columns": ["image", "text"],
        "index": [0],
        "data": [
            [
                base64.encodebytes(read_image(sample_image)).decode("utf-8"),
                labels,
            ],  # the labels are required in the first row of the "text" column
        ],
    }
}

# Create request json
request_file_name = "sample_request_data.json"
with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [ ]:
for i in range(6):
    # Score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
    response = workspace_ml_client.online_endpoints.invoke(
        endpoint_name="<online endpoint name>",
        deployment_name="<deployment name>",
        request_file=request_file_name,
    )
    print(f"raw response: {response}\n")

### 6. Clean up resources - delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()